In [2]:
import sys
sys.path.append('/home/s/seanmacb/Camera/pkgs')
import run_six_utils as rsu
import lsst.afw.display as afwDisplay
import lsst.daf.butler as daf_butler
afwDisplay.setDefaultBackend('matplotlib') 
import matplotlib.pyplot as plt
import numpy as np
import myutils_ar.myutils as myu
import eo_pipe

In [18]:
repo = '/repo/main'
operator = 'lsstccs'

repo_arr = ['/repo/main','/repo/main','/repo/ir2','/repo/ir2']

notable_detectors = ["R12_S20","R31_S01","R22_S01","R13_S01"]
defect_list = ["BRIGHT_COLUMNS","BRIGHT_PIXELS"]

kwarg_dict = [{"alpha":.9,"ls":"-","marker":"+","markersize":10,"color":"red"},
              {"alpha":.9,"ls":"--","marker":"x","markersize":10,"color":"blue"},
              {"alpha":.9,"ls":"-.","marker":".","markersize":10,"color":"green"}]

## Using utils in `myutils.py` to study specific amps

In [14]:
detector_amps = [("R11_S21",["C17"]),
                 ("R12_S20",["C17","C16"]),
                 ("R13_S01",["C17"]),
                 ("R13_S10",["C07"]),
                 ("R22_S01",["C11"]),
                 ("R23_S10",["C13"]),
                 ("R31_S01",["C05"]),
                 ("R32_S10",["C04"])]

run_week_repo = [('12845','w_2023_22','/repo/main'),
                 ('13162','w_2023_22','/repo/main'),
                 ('13401','w_2023_24','/repo/ir2'),
                 ('13550','w_2023_41','/repo/ir2')]  # Run 5, 6, run 6b.
weeklys = ['w_2023_22','w_2023_24','w_2023_41']
img_type = 'flat'
defect_datasets=["eoDarkDefects","eoBrightDefects"]
defect_dataset_path=["eo_dark_defects","eo_bright_defects"]

In [15]:
for det_amp in detector_amps:
    detector_str,amp_strs = det_amp
    detector_num = myu.det_nums[detector_str]
    for amp_str in amp_strs:
        # Make a new figure

        fig = 'mtv';
        plt.close(fig);
        fig = plt.figure(fig);

        # fig,axs = plt.subplots(1,len(run_week_repo),figsize=[5*len(run_week_repo)/3,5],sharey=True)
        fig,axs = plt.subplots(1,len(run_week_repo),figsize=[15,10],sharey=True)
        

        fig.suptitle(detector_str+" "+amp_str)
        
        for wkr,ax in zip(run_week_repo,axs):
            run,week,repo = wkr

            # Make a butler here in order to make image masks

            collections = 'LSSTCam/raw/all,LSSTCam/calib'.split(",") # Defining collections (that are ignored by below kernel)
            butler = daf_butler.Butler(repo,collections=collections) # Initializing a butler with the ir2 path and the above collections
            registry = butler.registry
            
            img, detector_obj, amp = rsu.getAmp(img_type,run,week,detector_str,amp_str,repo=repo)
            box,minx,maxx,miny,maxy = rsu.getAmpCorners(amp)

            # Mask image here

            kwargs={
                    "datasetType": "",
                    "collections": "",
                    "where":
                    """
                    instrument='LSSTCam' and 
                    detector = {var}
                    """.format(var=detector_num)
                    }

            for datasetType,defect_path in zip(defect_datasets,defect_dataset_path):
                img.getMask().addMaskPlane(datasetType)
                # here, add a new mask plane to add the mask.maskPixels to
                kwargs['datasetType'] = datasetType
                kwargs['collections'] = "u/lsstccs/"+defect_path+"_"+run+"_"+week
                # print("Querying",datasetType,"for mask")
                datasetRefs = list(registry.queryDatasets(**kwargs))
                mask = butler.get(datasetRefs[0])
                
                mask.maskPixels(img,maskName=datasetType)

            # add data to axes here

            disp = afwDisplay.Display(fig)
            disp.scale('asinh', 'zscale', Q=8)
            # if run==run_week_repo[-1][0]:
            
            rsu.formatter(disp,kw=defect_datasets)
            plt.sca(ax)
            disp.mtv(img)
            disp.show_colorbar(False)
            
            #
            
            # format general things here
            ax.set_title(run)
            ax.set_xlabel("Serial Register")
            # minx,maxx = 2400,2500
            # miny,maxy = 200,375
            ax.set_xlim(minx,maxx)
            ax.set_xticks(np.arange(minx,maxx,step=int(np.floor((maxx-minx)/2))))
            ax.set_ylim(miny,maxy)
            ax.set_yticks(np.arange(miny,maxy,step=int(np.floor((maxy-miny)/6))))

        axs[0].set_ylabel("Parallel Register")
        
        fig.tight_layout()
        
        fig.savefig("/home/s/seanmacb/Camera/Run6Comparisons/Figures/AmpStudy/inset_{}_{}.jpg".format(detector_str,amp_str),dpi=200)
        
        fig.clear()
        plt.close(fig)

<Figure size 640x480 with 0 Axes>